<a href="https://colab.research.google.com/github/trismegisto78/romeo/blob/master/testBidirLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
from random import random
from numpy import array
from numpy import cumsum
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import Flatten,CuDNNLSTM
from keras.initializers import glorot_uniform


Using TensorFlow backend.


In [2]:
!git clone https://github.com/trismegisto78/romeo

Cloning into 'romeo'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 51 (delta 23), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (51/51), done.


MOUNT GOOGLE DRIVE

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
with open('/content/gdrive/My Drive/strMacd2004.csv', 'r') as f:
  lerighe = f.read().split('\n')
del lerighe[0]

In [14]:
righe_file=[]
for k in range(0,1000000) :
  righe_file.append(float(lerighe[k].split(";")[1]) )
  
print(righe_file[0])



1.2596


In [0]:
# create a sequence classification instance
def get_sequence(dataset_array):
  idx = np.random.randint(0,len(dataset_array)-130  )
  X,y = get_sequence_from_idx(dataset_array,idx)
  return X,y

def get_sequence_from_idx(dataset_array,idx):
  
  X = np.zeros(n_timesteps)
  Y = np.zeros(1)
   
  
  for k in range(idx,idx+n_timesteps ):
    value=normValue(dataset_array[k])
    X[k-idx]=value
    
  finalValue=normValue(dataset_array[idx+130])

  if finalValue<(value-0.00050):
    Y[0]=1.0
  else:
    Y[0]=0.0
  
  X = X.reshape(n_timesteps,1)
  Y = Y.reshape(1)
  return X, Y

def get_a_batch(batchSize,data_set):
  totexamples = batchSize
  xInput = np.zeros( (totexamples,n_timesteps,1) )
  target = np.zeros( (totexamples,1) )
  for k in range(0,totexamples):
    X,y = get_sequence(data_set)
    xInput[k] = X
    target[k] = y
  
  return xInput,target

def get_simple_random_batch(batchSize,data_set):
  idx = np.random.randint(0,len(data_set),batchSize  )
  fullBatch = data_set[idx]
  
  xInput = np.zeros( (batchSize,n_timesteps,1) )
  target = np.zeros( (batchSize,1) )
  
  for k in range( len(fullBatch)  ):
    xInput[k] = fullBatch[k][0]
    target[k] = fullBatch[k][1]
  
  return xInput,target

def get_simple_all_dataset(data_set):
  
  xInput = np.zeros( (len(data_set),n_timesteps,1) )
  target = np.zeros( (len(data_set),1) )
  
  for k in range( len(data_set)  ):
    xInput[k] = data_set[k][0]
    target[k] = data_set[k][1]
  
  return xInput,target

def getAllDataSet(data_set):
  totexamples = len(data_set)-130
  xInput = np.zeros( (totexamples,n_timesteps,1) )
  target = np.zeros( (totexamples,1) )
  listone = []
  for k in range(0,totexamples):
    X,y = get_sequence_from_idx(data_set,k)
#    xInput[k] = X
#    target[k] = y
    listone.append([X,y])
    if k % 100000 ==0 :
      print('generate %d dataset items ' % k)
  return listone#xInput,target 


In [19]:
ilMax = max(righe_file)
ilMin = min(righe_file)


delta = ilMax-ilMin


print('MAX ', ilMax )
print('MIN ', ilMin )
print('delta ', delta )

MAX  1.3663
MIN  1.164
delta  0.20230000000000015


In [0]:
def normValue(theValue):
  return (float(theValue)-ilMin)/delta


In [22]:
from sklearn.model_selection import train_test_split

n_timesteps = 100

allDataset = getAllDataSet(righe_file)


train, validation = train_test_split(allDataset,shuffle=False)

print("TRAIN LEN ",len(train) )
print("validation LEN ",len(validation) )


generate 0 dataset items 
generate 100000 dataset items 
generate 200000 dataset items 
generate 300000 dataset items 
generate 400000 dataset items 
generate 500000 dataset items 
generate 600000 dataset items 
generate 700000 dataset items 
generate 800000 dataset items 
generate 900000 dataset items 
TRAIN LEN  749902
validation LEN  249968


In [0]:
#train[0].head()
array_train = np.asarray(train )
array_val = np.asarray(validation )



In [24]:


model = Sequential()
model.add(Bidirectional(CuDNNLSTM(n_timesteps, return_sequences=True,kernel_initializer=glorot_uniform()), 
                        input_shape=(n_timesteps, 1), name='inputBIDI'))
model.add(Bidirectional(CuDNNLSTM(256, return_sequences=True,kernel_initializer=glorot_uniform())))
model.add(TimeDistributed(Dense(1, activation='tanh')))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid')  ) 

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputBIDI (Bidirectional)    (None, 100, 200)          82400     
_________________________________________________________________
bidirectional_8 (Bidirection (None, 100, 512)          937984    
_________________________________________________________________
time_distributed_7 (TimeDist (None, 100, 1)            513       
_________________________________________________________________
flatten_5 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 101       
Total params: 1,020,998
Trainable params: 1,020,998
Non-trainable params: 0
_________________________________________________________________


In [0]:
samplePos=0
# train LSTM

xInput,target = get_simple_all_dataset(validation)

for epoch in range(1000):
 
  X,y = get_simple_random_batch(750,array_train )
  resTrain = model.train_on_batch(X,y)

  if(epoch % 10 == 0):
    print('Apprendimento epoch->',epoch,' ',resTrain)
    
  if(epoch % 1000 == -1 ):
    totexamples = 1000
    resulto = model.evaluate(xInput, target, verbose=1,batch_size=512)

    print('Eseguo validazione %.2f%%'%(resulto[1]*100), '>>>>>',resulto  )
    print('')

  
# evaluate LSTM



Apprendimento epoch-> 0   [0.68052906, 0.5786667]
Apprendimento epoch-> 10   [0.6789285, 0.58933336]
Apprendimento epoch-> 20   [0.67785275, 0.588]
Apprendimento epoch-> 30   [0.68013966, 0.58133334]
Apprendimento epoch-> 40   [0.6697282, 0.60933334]
Apprendimento epoch-> 50   [0.6749777, 0.596]
Apprendimento epoch-> 60   [0.678522, 0.58533335]
Apprendimento epoch-> 70   [0.67774373, 0.588]
Apprendimento epoch-> 80   [0.6858933, 0.56133336]
Apprendimento epoch-> 90   [0.6727361, 0.604]
Apprendimento epoch-> 100   [0.68875885, 0.55866665]
Apprendimento epoch-> 110   [0.6906148, 0.55333334]
Apprendimento epoch-> 120   [0.67031115, 0.60933334]
Apprendimento epoch-> 130   [0.6705661, 0.6066667]
Apprendimento epoch-> 140   [0.67031777, 0.6066667]
Apprendimento epoch-> 150   [0.6772576, 0.58933336]
Apprendimento epoch-> 160   [0.68237215, 0.5746667]
Apprendimento epoch-> 170   [0.6811555, 0.57733333]
Apprendimento epoch-> 180   [0.6697227, 0.6106667]
Apprendimento epoch-> 190   [0.68290365, 